# Model recommendation with lighfm

### Import libraries

In [1]:
import os
import sys
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy
import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from lightfm import cross_validation
import scipy.sparse as sp
from scipy import sparse

In [2]:
from lightfm import LightFM
from sklearn.base import clone


class LightFMResizable(LightFM):
    """A LightFM that resizes the model to accomodate new users,
    items, and features"""

    def fit_partial(
        self,
        interactions,
        user_features=None,
        item_features=None,
        sample_weight=None,
        epochs=1,
        num_threads=1,
        verbose=False,
    ):
        try:
            self._check_initialized()
            self._resize(interactions, user_features, item_features)
        except ValueError:
            # This is the first call so just fit without resizing
            pass

        super().fit_partial(
            interactions,
            user_features,
            item_features,
            sample_weight,
            epochs,
            num_threads,
            verbose,
        )

        return self

    def _resize(self, interactions, user_features=None, item_features=None):
        """Resizes the model to accommodate new users/items/features"""

        no_components = self.no_components
        no_user_features, no_item_features = interactions.shape  # default

        if hasattr(user_features, "shape"):
            no_user_features = user_features.shape[-1]
        if hasattr(item_features, "shape"):
            no_item_features = item_features.shape[-1]

        if (
            no_user_features == self.user_embeddings.shape[0]
            and no_item_features == self.item_embeddings.shape[0]
        ):
            return self

        new_model = clone(self)
        new_model._initialize(no_components, no_item_features, no_user_features)

        # update all attributes from self._check_initialized
        for attr in (
            "item_embeddings",
            "item_embedding_gradients",
            "item_embedding_momentum",
            "item_biases",
            "item_bias_gradients",
            "item_bias_momentum",
            "user_embeddings",
            "user_embedding_gradients",
            "user_embedding_momentum",
            "user_biases",
            "user_bias_gradients",
            "user_bias_momentum",
        ):
            # extend attribute matrices with new rows/cols from
            # freshly initialized model with right shape
            old_array = getattr(self, attr)
            old_slice = [slice(None, i) for i in old_array.shape]
            new_array = getattr(new_model, attr)
            new_array[tuple(old_slice)] = old_array
            setattr(self, attr, new_array)

        return self

### Defining variables

In [3]:
import json

with open('config.json', 'r') as f:
    config = json.load(f)

In [4]:
# percentage of data used for testing
TEST_PERCENTAGE = 0.20
# model learning rate
LEARNING_RATE = 0.1
# no of epochs to fit model
NO_EPOCHS = 80

NUM_COMPONENTS = 30

NUM_THREADS = 4

ALPHA = 1e-3

MAX_SAMPLED = 10

# seed for pseudonumber generations
SEED = 42

### Retrieve data

In [5]:
dtype_df_train_score = {
"userId" : 'string',
"userType" : 'category',
"history" : 'string',
"score" : 'Float32'
}

In [6]:
import pandas as pd

# df_ratings = pd.read_csv(config["DF_TRAIN_SCORES"], dtype=dtype_df_train_score)
df_ratings = pd.read_csv(config["DF_TRAIN_SCORES"], dtype=dtype_df_train_score, nrows=500000)
df_ratings

,Unnamed: 0,userId,history,userType,score
0,0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,80aa7bb2-adce-4a55-9711-912c407927a1,Non-Logged,2.216477
1,1,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,d9e5f15d-b441-4d8b-bee4-462b106d3916,Non-Logged,2.429501
2,2,17f1083e6079b0f28f7820a6803583d1c1b405c0718b11...,e273dba4-136c-45fb-bdd6-0cc57b13aaf0,Non-Logged,1.794861
3,3,528a8d7a2af73101da8d6709c1ec875b449a5a58749a99...,a0562805-c7d1-4ffd-b622-87c50ae006f4,Non-Logged,1.68271
4,4,2dd18b58a634a4e77181a202cf152df6169dfb3e4230ef...,233f8238-2ce0-470f-a9d5-0e0ac530382a,Non-Logged,2.266852
...,...,...,...,...,...
499995,499995,8f8bf43e20c6456d79e628509ebd808a19fe55bf08fe01...,f3e8e1c1-b9eb-4408-acae-6de19135e246,Non-Logged,0.360503
499996,499996,8f8bf43e20c6456d79e628509ebd808a19fe55bf08fe01...,3a6a9c38-e7ae-437c-86bf-054d49a5ae80,Non-Logged,0.335948
499997,499997,8f8bf43e20c6456d79e628509ebd808a19fe55bf08fe01...,fd15abfd-2015-4faa-8d97-61a6531bdc5f,Non-Logged,0.335448
499998,499998,8f8bf43e20c6456d79e628509ebd808a19fe55bf08fe01...,f788745b-b539-47bb-88f2-a92da5bcbd80,Non-Logged,0.269779


In [7]:
df_ratings.drop(columns=["Unnamed: 0"],inplace=True)
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   userId    500000 non-null  string  
 1   history   500000 non-null  string  
 2   userType  500000 non-null  category
 3   score     500000 non-null  Float32 
dtypes: Float32(1), category(1), string(2)
memory usage: 10.5 MB


In [8]:
df_ratings.describe()

,score
count,500000.0
mean,1.626204
std,0.865815
min,0.037463
25%,0.948389
50%,1.590375
75%,2.218244
max,4.902971


In [9]:
dtype_df_items = {
"page" : 'string',
"url" : 'string',
"issued" : 'string',
"modified" : 'string',
"title" : 'string',
"body" : 'string',
"caption" : 'string',
"age_in_days" : 'UInt32',
"age_exp" : 'Float32',
"age_exp_normalized" : 'Float32',
"ageCategories" : 'category'
}

In [10]:
df_news = pd.read_csv(config["DF_ITEMS_FEATURE"], dtype=dtype_df_items)
df_news.drop(columns=["Unnamed: 0"],inplace=True)

In [11]:
df_news.head(3)

,page,age_exp_normalized,ageCategories
0,7371a9b5-5824-4c57-8704-00a74feebe79,0.151439,very-old
1,7a5ea08f-4583-49e2-ba52-a71999443f7b,0.140788,very-old
2,6afc8bbb-4f36-43d5-8a44-a2917df5621a,0.12261,very-old


In [12]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255603 entries, 0 to 255602
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   page                255603 non-null  string  
 1   age_exp_normalized  255603 non-null  Float32 
 2   ageCategories       255603 non-null  category
dtypes: Float32(1), category(1), string(1)
memory usage: 3.4 MB


In [13]:
df_merged = pd.merge(df_ratings, df_news, left_on='history', right_on='page', how='left')
df_merged.drop(columns=["page"],inplace=True) # 'page' is the same as 'history'

In [14]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   userId              500000 non-null  string  
 1   history             500000 non-null  string  
 2   userType            500000 non-null  category
 3   score               500000 non-null  Float32 
 4   age_exp_normalized  500000 non-null  Float32 
 5   ageCategories       500000 non-null  category
dtypes: Float32(2), category(2), string(2)
memory usage: 13.4 MB


In [15]:
df_merged.head()

,userId,history,userType,score,age_exp_normalized,ageCategories
0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,80aa7bb2-adce-4a55-9711-912c407927a1,Non-Logged,2.216477,0.980416,recent
1,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,d9e5f15d-b441-4d8b-bee4-462b106d3916,Non-Logged,2.429501,0.613061,mid
2,17f1083e6079b0f28f7820a6803583d1c1b405c0718b11...,e273dba4-136c-45fb-bdd6-0cc57b13aaf0,Non-Logged,1.794861,0.880859,recent
3,528a8d7a2af73101da8d6709c1ec875b449a5a58749a99...,a0562805-c7d1-4ffd-b622-87c50ae006f4,Non-Logged,1.68271,0.945895,recent
4,2dd18b58a634a4e77181a202cf152df6169dfb3e4230ef...,233f8238-2ce0-470f-a9d5-0e0ac530382a,Non-Logged,2.266852,0.13293,very-old


In [16]:
w_initial_score = 0.85
w_age_norm = 1-w_initial_score
df_merged["score"] = df_merged["score"]*w_initial_score + df_merged["age_exp_normalized"]*w_age_norm
df_merged["score"] = df_merged["score"]/df_merged["score"].max()
df_merged.describe()

,score,age_exp_normalized
count,500000.0,500000.0
mean,0.351594,0.90056
std,0.171273,0.184292
min,0.014885,0.103527
25%,0.217587,0.934135
50%,0.34469,0.956106
75%,0.469085,0.978657
max,1.0,1.0


In [17]:
df_merged = df_merged[["userId","history","userType","score"]]

df_merged.rename(columns={"ageCategories" : "historyFreshnessNormalized"},inplace=True)
df_merged.describe()

,score
count,500000.0
mean,0.351594
std,0.171273
min,0.014885
25%,0.217587
50%,0.34469
75%,0.469085
max,1.0


In [18]:
# df_merged.head()
df_merged.head()

,userId,history,userType,score
0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,80aa7bb2-adce-4a55-9711-912c407927a1,Non-Logged,0.470629
1,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,d9e5f15d-b441-4d8b-bee4-462b106d3916,Non-Logged,0.499817
2,17f1083e6079b0f28f7820a6803583d1c1b405c0718b11...,e273dba4-136c-45fb-bdd6-0cc57b13aaf0,Non-Logged,0.384128
3,528a8d7a2af73101da8d6709c1ec875b449a5a58749a99...,a0562805-c7d1-4ffd-b622-87c50ae006f4,Non-Logged,0.364299
4,2dd18b58a634a4e77181a202cf152df6169dfb3e4230ef...,233f8238-2ce0-470f-a9d5-0e0ac530382a,Non-Logged,0.451094


In [19]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   userId    500000 non-null  string  
 1   history   500000 non-null  string  
 2   userType  500000 non-null  category
 3   score     500000 non-null  Float32 
dtypes: Float32(1), category(1), string(2)
memory usage: 10.5 MB


### Prepare data

Before fitting the LightFM model, we need to create an instance of `Dataset` which holds the interaction matrix.

In [20]:
dataset = Dataset()

# Get unique values for users, items, and user features
unique_users = df_merged["userId"].unique()
unique_items = df_merged["history"].unique()
unique_user_features = df_merged["userType"].unique().tolist()

# Fit dataset with users, items, and user feature names
dataset.fit(
    users=unique_users,
    items=unique_items,
    user_features=unique_user_features  # Register user features
)


In [21]:
(interactions, weights) = dataset.build_interactions([
    (row.userId, row.history, row.score) 
    for _, row in df_merged.iterrows()
])


In [22]:
user_features_list = [
    (row.userId, [row.userType])  
    for _, row in df_merged.iterrows()
]

user_features = dataset.build_user_features(user_features_list)


LightLM works slightly differently compared to other packages as it expects the train and test sets to have same dimension. Therefore the conventional train test split will not work.

The package has included the `cross_validation.random_train_test_split` method to split the interaction data and splits it into two disjoint training and test sets. 

However, note that **it does not validate the interactions in the test set to guarantee all items and users have historical interactions in the training set**. Therefore this may result into a partial cold-start problem in the test set.

In [23]:
# Split train and test sets (80/20 split)
train, test = cross_validation.random_train_test_split(interactions, test_percentage=TEST_PERCENTAGE, random_state=SEED)
train_weights, test_weights  = cross_validation.random_train_test_split(weights, test_percentage=TEST_PERCENTAGE, random_state=SEED)


Double check the size of both the train and test sets.

In [24]:
print(f"Shape of train interactions: {train.shape}")
print(f"Shape of test interactions: {test.shape}")

Shape of train interactions: (41871, 63893)
Shape of test interactions: (41871, 63893)


In [25]:
print(f"Shape of train interactions: {train_weights.shape}")
print(f"Shape of test interactions: {test_weights.shape}")

Shape of train interactions: (41871, 63893)
Shape of test interactions: (41871, 63893)


### Fit the LightFM model

In this notebook, the LightFM model will be using the weighted Approximate-Rank Pairwise (WARP) as the loss. Further explanation on the topic can be found [here](https://making.lyst.com/lightfm/docs/examples/warp_loss.html#learning-to-rank-using-the-warp-loss).


In general, it maximises the rank of positive examples by repeatedly sampling negative examples until a rank violation has been located. This approach is recommended when only positive interactions are present.

The LightFM model can be fitted with the following code:

In [26]:
model = LightFMResizable(no_components=NUM_COMPONENTS,loss="warp",learning_rate=LEARNING_RATE,user_alpha=ALPHA,max_sampled=MAX_SAMPLED,random_state=np.random.RandomState(SEED))  # Weighted Approximate-Rank Pairwise (WARP) loss
model.fit(train, sample_weight=train_weights, epochs=NO_EPOCHS, num_threads=NUM_THREADS, user_features=user_features)
# model.fit(train, epochs=NO_EPOCHS, num_threads=NUM_THREADS, user_features=user_features)
# model.fit(interactions, sample_weight=weights, epochs=NO_EPOCHS, num_threads=NUM_THREADS, user_features=user_features)


### Evaluate model

In [27]:
# Import the evaluation routines
from lightfm.evaluation import auc_score

# Compute evaluation metrics
auc_train = auc_score(model, train, user_features=user_features, num_threads=NUM_THREADS).mean()
auc_test = auc_score(model, test, train_interactions=train, user_features=user_features, num_threads=NUM_THREADS).mean()

# Print evaluation results
print(f"AUC test Score: {auc_test:.4f}")
print(f"AUC train Score: {auc_train:.4f}")

AUC test Score: 0.7527
AUC train Score: 0.8325


### Save pkls to serve model

In [28]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

In [29]:
item_id_map_reverse = {v: k for k, v in item_id_map.items()}

In [30]:
import pickle

pickle.dump(model, open('artifacts/lightfm_model.pkl', 'wb'))
pickle.dump(user_id_map, open('artifacts/user_id_map.pkl', 'wb'))
pickle.dump(item_id_map_reverse, open('artifacts/item_id_map_reverse.pkl', 'wb'))
pickle.dump(user_feature_map, open('artifacts/user_feature_map.pkl', 'wb'))

In [31]:
loaded_model = pickle.load(open('artifacts/lightfm_model.pkl', 'rb'))
loaded_user_id_map = pickle.load(open('artifacts/user_id_map.pkl', 'rb'))
loaded_item_id_map_reverse = pickle.load(open('artifacts/item_id_map_reverse.pkl', 'rb'))
loaded_user_feature_map = pickle.load(open('artifacts/user_feature_map.pkl', 'rb'))

### Make predictions to known and unknowm on same recommendation function with pkls

In [32]:
interactions.shape

(41871, 63893)

In [33]:

_, n_items = interactions.shape # no of users * no of items

n_items

63893

In [34]:
def format_newuser_input(user_feature_map, user_feature_list):
  normalised_val = 1.0 
  target_indices = []
  for feature in user_feature_list:
    try:
        target_indices.append(user_feature_map[feature])
    except KeyError:
        print("new user feature encountered '{}'".format(feature))
        pass
  #print("target indices: {}".format(target_indices))
  new_user_features = np.zeros(len(user_feature_map.keys()))
  for i in target_indices:
    new_user_features[i] = normalised_val
  new_user_features = sparse.csr_matrix(new_user_features)
  return(new_user_features)

In [35]:
def sample_recommendation(user_hash,df_news,user_feature_list,item_id_map_reverse,user_feature_map,user_id_map,model):
    try:
        user_x = user_id_map[user_hash]
        scores = model.predict(user_x, np.arange(n_items)) # means predict for all
    except:
        new_user_features = format_newuser_input(user_feature_map, user_feature_list)
        scores = model.predict(0, np.arange(n_items), user_features=new_user_features)
    
    top_5_indices = np.argsort(-scores)[:5]  # Sort scores in descending order and take the top 5
    top_5_items = [item_id_map_reverse[i] for i in top_5_indices]

    print("Top 5 recommended items:")

    for x in top_5_items:
        row = df_news[df_news["page"] == x]
        print("        %s" % row["page"].values[0])


In [36]:
# # predict for known user
# user_feature_list = ['userType:Logged']
# user_hash = '5f5e17781fc2ec0ddcfb2e9356e61c5d3d4b0b3c8fabd20917feb9e807463856'
# sample_recommendation(user_hash,df_news,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)

In [37]:
# predict for unknown user
user_feature_list = ['userType:Non-Logged']
user_hash = ''
sample_recommendation(user_hash,df_news,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)

new user feature encountered 'userType:Non-Logged'
Top 5 recommended items:
        d2593c3d-2347-40d9-948c-b6065e8459a9
        f6b5d170-48b9-4f8e-88d4-c84b6668f3bd
        bf257382-74fb-4392-ad6a-143240e39f81
        1f32787b-de2b-49be-8c20-ddaeae34cc22
        f0a78e58-ec7e-494c-9462-fbd6446a9a89
